# Lecon 3 : La memoire du modele

## Le probleme de la memoire courte

Dans les lecons precedentes, notre modele ne regardait que la **derniere lettre**.
C'est comme essayer de deviner la fin d'une phrase en n'ecoutant que le dernier mot.

Exemple : apres la lettre 'a', le modele ne sait pas si on est dans
"**cla**ra" ou "**a**dam" -- pourtant la suite est tres differente !

Solution : donner une **memoire** au modele. On appelle ca les **embeddings**.

## Les embeddings : transformer des lettres en nombres

L'idee : chaque lettre est representee par une **liste de nombres** (un vecteur).

Par exemple :
- 'a' -> [0.3, -0.1, 0.8]
- 'b' -> [-0.5, 0.4, 0.2]

Ces nombres ne sont pas choisis a la main : le modele les **apprend** pendant
l'entrainement. Les lettres qui se comportent de facon similaire auront
des nombres proches.

In [ ]:
import random
import math

# Notre alphabet
alphabet = list(".abcdefghijklmnopqrstuvwxyz")
char_to_id = {c: i for i, c in enumerate(alphabet)}
id_to_char = {i: c for i, c in enumerate(alphabet)}
vocab_size = len(alphabet)

print(f"Taille du vocabulaire : {vocab_size} caracteres")
print(f"Exemples : 'a' = {char_to_id['a']}, 'z' = {char_to_id['z']}, '.' = {char_to_id['.']}")

In [ ]:
# Creons les embeddings : chaque lettre = un vecteur de taille 8
EMBED_DIM = 8

random.seed(42)

# Initialisation aleatoire (le modele apprendra les bonnes valeurs)
embeddings = [
    [random.gauss(0, 0.5) for _ in range(EMBED_DIM)]
    for _ in range(vocab_size)
]

print(f"Embedding de 'a' : {[f'{x:.2f}' for x in embeddings[char_to_id['a']]]}")
print(f"Embedding de 'b' : {[f'{x:.2f}' for x in embeddings[char_to_id['b']]]}")
print()
print("Pour l'instant ces nombres sont aleatoires.")
print("Apres entrainement, les lettres similaires auront des vecteurs proches.")

## Regarder plusieurs lettres en arriere

Maintenant, au lieu de regarder 1 seule lettre, on va regarder les
**3 dernieres lettres** (notre "fenetre de contexte").

On **concatene** (met bout a bout) leurs embeddings pour avoir une image
complete du contexte.

In [ ]:
CONTEXT_SIZE = 3  # On regarde 3 lettres en arriere

def get_context_vector(mot, position, embeddings):
    """Recupere les embeddings des 3 dernieres lettres et les concatene."""
    vecteur = []
    for i in range(CONTEXT_SIZE):
        pos = position - CONTEXT_SIZE + i
        if pos < 0:
            # Avant le debut du mot, on utilise le padding (.)
            char_id = char_to_id['.']
        else:
            char_id = char_to_id[mot[pos]]
        vecteur.extend(embeddings[char_id])
    return vecteur

# Exemple : pour predire la 4e lettre de "hugo"
mot = ".hugo."
position = 4  # on veut predire 'o' (position 4)
contexte = get_context_vector(mot, position, embeddings)

print(f"Mot : '{mot}'")
print(f"Pour predire la lettre en position {position} ('{mot[position]}'),")
print(f"on regarde les 3 lettres precedentes : '{mot[position-3:position]}'")
print(f"Vecteur de contexte : {len(contexte)} nombres ({CONTEXT_SIZE} x {EMBED_DIM})")

## Un mini reseau de neurones

On prend le vecteur de contexte et on le passe dans un **reseau de neurones**
simple (une seule couche) pour obtenir les probabilites de chaque lettre.

```
[contexte: 24 nombres] --> [couche: multiplication + addition] --> [27 scores] --> [probas]
```

In [ ]:
INPUT_DIM = CONTEXT_SIZE * EMBED_DIM  # 3 * 8 = 24

# Les poids de notre couche (une matrice 24 x 27)
W = [[random.gauss(0, 0.3) for _ in range(vocab_size)] for _ in range(INPUT_DIM)]
b = [0.0] * vocab_size  # biais

def forward(contexte, W, b):
    """Passe le contexte dans le reseau pour obtenir des scores."""
    scores = list(b)  # copie du biais
    for j in range(vocab_size):
        for i in range(INPUT_DIM):
            scores[j] += contexte[i] * W[i][j]
    return scores

def softmax(scores):
    """Transforme les scores en probabilites (entre 0 et 1, somme = 1)."""
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]

# Test
scores = forward(contexte, W, b)
probas = softmax(scores)

# Top 5 predictions pour la lettre apres 'hug' dans 'hugo'
top5 = sorted(range(vocab_size), key=lambda i: -probas[i])[:5]
print("Predictions (avant entrainement) pour la lettre apres 'hug' :")
for idx in top5:
    print(f"  '{id_to_char[idx]}' : {probas[idx]:.1%}")
print("\n  (C'est du hasard pour l'instant - il faut entrainer !)")

In [ ]:
# Entrainement
prenoms = [
    "emma", "lucas", "lea", "hugo", "chloe",
    "louis", "alice", "jules", "lina", "adam",
    "rose", "arthur", "manon", "paul", "jade",
    "nathan", "eva", "leo", "clara", "noah",
]

vitesse = 0.01

print("Entrainement avec contexte de 3 lettres...")
print()

for epoch in range(100):
    loss_totale = 0
    nb = 0

    for prenom in prenoms:
        mot = "." + prenom + "."
        for pos in range(1, len(mot)):
            cible = char_to_id[mot[pos]]

            # Forward
            ctx = get_context_vector(mot, pos, embeddings)
            scores = forward(ctx, W, b)
            probas = softmax(scores)

            # Loss
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1

            # Gradient simplifie pour W et b
            for j in range(vocab_size):
                grad = probas[j] - (1 if j == cible else 0)
                b[j] -= vitesse * grad
                for i in range(INPUT_DIM):
                    W[i][j] -= vitesse * grad * ctx[i]

    if epoch % 20 == 0:
        print(f"  Epoch {epoch:3d} | Loss : {loss_totale / nb:.3f}")

print(f"  Epoch {epoch:3d} | Loss : {loss_totale / nb:.3f}")

In [ ]:
# Generer avec le modele entraine
def generer(n=10):
    resultats = []
    for _ in range(n):
        mot = "."
        for _ in range(20):
            ctx = get_context_vector(mot, len(mot), embeddings)
            scores = forward(ctx, W, b)
            probas = softmax(scores)
            idx = random.choices(range(vocab_size), weights=probas, k=1)[0]
            if idx == char_to_id['.']:
                break
            mot += id_to_char[idx]
        if len(mot) > 1:
            resultats.append(mot[1:].capitalize())
    return resultats

print("Prenoms generes (avec contexte de 3 lettres) :")
print()
for p in generer(10):
    print(f"  {p}")

print()
print("Mieux qu'avant ! Le modele 'comprend' des combinaisons de lettres.")

## Ce qu'on a appris

- Les **embeddings** transforment des lettres en nombres que le modele peut manipuler
- Un **contexte** plus large (3 lettres au lieu de 1) donne de meilleurs resultats
- Un **reseau de neurones** (meme simple) combine le contexte pour faire des predictions

### Et ensuite ?

Notre modele regarde toujours une fenetre fixe de 3 lettres. Et s'il pouvait
**choisir** quelles lettres sont importantes, meme si elles sont loin ?
C'est exactement ce que fait le **mecanisme d'attention** -- le coeur des GPT.

---
*Prochaine lecon : [04 - L'attention](04_lattention.ipynb)*